In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering

import holidays
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np





In [5]:
import warnings
import requests
#import pvlib
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import json
import gc
import datetime
from tqdm.notebook import tqdm
from sklearn.preprocessing import SplineTransformer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error
#from pvlib.location import Location
#from pvlib import clearsky, solarposition,tracking
#from meteostat import Hourly,Point
from datetime import datetime
from catboost import CatBoostRegressor

tqdm.pandas()
warnings.filterwarnings("ignore")

#pd.set_option('display.max_columns', 300)
#pd.set_option('display.max_rows', 300)


In [6]:
class CFG:
    datetime_features=True
    outlier_detect = True
    extract_features  = True
    extract_features2 = False
    deal_DATE = True
    seasonality_spline_features = True
    lag_feats = False
    lat_lon = False
    beijing_latlon = False
    corr_features_transform = False
    target = 'power'
    train_path = './data/Rawdata/train_df.csv'
    test_path = './data/Rawdata/test_df.csv'
    submission_path = './data/Rawdata/submission.csv'
    
    

In [7]:
train_df = pd.read_csv(CFG.train_path)
test_df = pd.read_csv(CFG.test_path)

submission = pd.read_csv(CFG.submission_path)

train_df.rename(columns={"DATE":"date","POWER":"power"},inplace=True)
test_df.rename(columns={"DATE":"date"},inplace=True)



train_df.drop(columns=['SOT',"ID"], axis=1, inplace=True)
test_df.drop(columns=['SOT',"ID"], axis=1, inplace=True)

In [8]:
def datetime_features(df):
    
    col = "date"
    df['month'] = df[col].dt.month
    df['hour'] = df[col].dt.hour
    df['year'] = df[col].dt.year
    df['dayofweek'] = df[col].dt.dayofweek
    df['quarter'] = df[col].dt.quarter
    df['dayofmonth'] = df[col].dt.day
    df['weekofyear'] = df[col].dt.weekofyear

    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday+1
    df['Monday'] = np.where(df['weekday']==1,1,0)
    df['Tuesday'] = np.where(df['weekday']==2,1,0)
    df['Wednesday'] = np.where(df['weekday']==3,1,0)
    df['Thursday'] = np.where(df['weekday']==4,1,0)
    df['Friday'] = np.where(df['weekday']==5,1,0)
 

    df['ymd'] = df['year']*10000+df['month']*100+df['day']
    df['sin'] = np.sin(2 * np.pi * df['ymd']/365)
    df['cos'] = np.cos(2 * np.pi * df['ymd']/365)
    
    return df

In [9]:
def outlier_detect(train_df,test_df):
    train_df = train_df[train_df['V_2'] > -6]
    train_df = train_df[train_df['V_4'] > 250]
    train_df = train_df[train_df['V_7'] >0.675]
    train_df = train_df[train_df['V_11'] < 14]
    train_df = train_df[train_df['V_12'] < 9]
    #Creating feature variables by four arithmetic operations.
    train_df['V1_2_6'] = train_df['V_1']+train_df['V_2']+train_df['V_6']
    train_df['V3_4_5'] = train_df['V_3']+train_df['V_4']+train_df['V_5']
    test_df['V1_2_6'] = test_df['V_1']+test_df['V_2']+test_df['V_6']
    test_df['V3_4_5'] = test_df['V_3']+test_df['V_4']+test_df['V_5']
    return train_df,test_df
    

In [10]:
def extract_features(train_df,test_df):
    train_df['1_/_V_1'] = train_df['V_1'].apply(lambda x: 1/(x + 1e-3))
    test_df['1_/_V_1'] = test_df['V_1'].apply(lambda x: 1/(x + 1e-3))
    train_df['V_1_square_2'] = train_df['V_1'].apply(lambda x: x**2)
    test_df['V_1_square_2'] = test_df['V_1'].apply(lambda x: x**2)
    return train_df,test_df

In [11]:
train_df.columns

Index(['date', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'V_11', 'V_12', 'V_13', 'LAT', 'LON', 'power'],
      dtype='object')

In [12]:
def extract_features2(train_df,test_df):
    for col in ["V_11","V_13","V_7","V_4","V_5","V_8","V_3","V3_4_5"]:
        train_df["inv"+col] = train_df[col].apply(lambda x: 1/(x + 1e-3))
        test_df["inv"+col] = test_df[col].apply(lambda x: 1/(x + 1e-3))

    return train_df,test_df


In [13]:
def deal_DATE(df):
    #传入的是一个有“DATE”这一列的DateFrame对象
    date=df['date'].values
    #将日期按照是否是工作日来分
    isweekday=[]
    for i in range(len(date)):
        #strptime:按特定的格式将字符串.weekday用来判断是星期几,用0~6表示星期一到星期日
        isweekday.append(int(datetime.strptime(date[i],'%Y/%m/%d').weekday()<5))#is weekday
    
    #将日期按星期几划分
    Mon=[0 for i in range(len(date))]
    Tue=[0 for i in range(len(date))]
    Wed=[0 for i in range(len(date))]
    Thur=[0 for i in range(len(date))]
    Fri=[0 for i in range(len(date))]
    Sat=[0 for i in range(len(date))]
    Sun=[0 for i in range(len(date))]
    
    for i in range(len(date)):
        #用0~6表示星期一到星期日
        num_of_the_week=int(datetime.strptime(date[i],'%Y/%m/%d').weekday())
        if num_of_the_week==0:
            Mon[i]=1
        elif num_of_the_week==1:
            Tue[i]=1
        elif num_of_the_week==2:
            Wed[i]=1
        elif num_of_the_week==3:
            Thur[i]=1
        elif num_of_the_week==4:
            Fri[i]=1
        elif num_of_the_week==5:
            Sat[i]=1
        else:# num_of_the_week==6
            Sun[i]=1
    
    
    #将日期按照季节来进行划分
    spring=[0 for i in range(len(date))]
    summer=[0 for i in range(len(date))]
    fall=[0 for i in range(len(date))]
    winter=[0 for i in range(len(date))]
    for i in range(len(date)):
        month=int(date[i].split("/")[1])
        if 2<month<6:#3,4,5
            spring[i]=1
        elif 5<month<9:#6,7,8
            summer[i]=1
        elif 8<month<12:#9,10,11
            fall[i]=1
        else:#12,1,2
            winter[i]=1 
    #在数据中加入特征(这是一年的第几周),并进行归一化操作.
    week=[]
    for i in range(len(date)):
        new_date = datetime.strptime(date[i], '%Y/%m/%d')
        week_number = new_date.isocalendar()[1]
        week.append(week_number/52)
    #将日期按照时间的先后顺序来分,具体方法是计算距离2016年1月1日的秒数
    distance=[]
    for i in range(len(date)):
        y,m,d=date[i].split("/")
        current_date=datetime(int(y), int(m), int(d), 0, 0, 0)
        # 定义2016年1月1日的日期时间
        epoch = datetime(2016, 1, 1, 0, 0, 0)#这里将2016换成2017试试
        # 计算日期距离2016年1月1日有多久，单位为秒
        delta = current_date - epoch
        distance.append(delta.total_seconds())
    #由于数据太大,可以对distance进行归一化处理
    min_dis=np.min(distance)#最小值
    max_dis=np.max(distance)#最大值
    normalize_dis=(distance-min_dis)/(max_dis-min_dis)
    
    
    df['week']=week
    df['Mon']=Mon
    df['Tue']=Tue
    df['Wed']=Wed
    df['Thur']=Thur
    df['Fri']=Fri
    df['Sat']=Sat
    df['Sun']=Sun
    
    df['distance']=normalize_dis
    df['isweekday']=isweekday
    df['spring']=spring
    df['summer']=summer
    df['fall']=fall
    df['winter']=winter
    
    
    #df.drop(['DATE'],axis=1,inplace=True)
    
    return df

In [14]:
def lag_features(df_temp,
                 columns,
                 lags):
    
    """
    Lag feature üreten fonksiyon, ilgili dataframe, lag uzunlukluları ve sütun isimleri verilerek lag'li featurelar üretilir.
    """
    for col in columns:
        for lag in lags:
            df_temp[f'lag_{lag}_{col}'] = df_temp[col].rolling(lag).mean().reset_index(drop=True)
    return df_temp


lag_feats_list = ['V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9',
       'V_10', 'V_11', 'V_12', 'V_13',]


In [15]:
def lag_feats(train_df,test_df):
    num=15
    df_merge = train_df.append(test_df).reset_index(drop=True)
    df_merge_pre = lag_features(df_merge,
                     lag_feats_list,
                     [num]).reset_index(drop=True)
    train_df = df_merge_pre.iloc[:train_df.shape[0]]
    test_df = df_merge_pre.iloc[train_df.shape[0]:].drop(columns=["power"])
    train_df = train_df.iloc[num:]
    return train_df,test_df

In [16]:
def lat_lon(df):
    # Calculating Distances Between Areas
    china_mtx = cdist(df[['LAT', 'LON']], df[['LAT', 'LON']], metric='euclidean')

    # clustering
    clustering = AgglomerativeClustering(n_clusters=10, linkage='single').fit(china_mtx)
    df['cluster'] = clustering.labels_
    return df


In [17]:
def beijing_latlon(df):
# Calculating distance from Beijing
    beijing_latlon = [39.9, 116.4]

    china_mtx = cdist(df[['LAT', 'LON']], [beijing_latlon], metric='euclidean')

    # clustering
    clustering = AgglomerativeClustering(n_clusters=12, linkage='single').fit(china_mtx)
    df['cluster_capital'] = clustering.labels_
    return df 

In [18]:
def corr_features_transform(df):
        
    df_cluster = df[['V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6','cluster']]
    df_1 = df_cluster.groupby('cluster').agg({'mean','std','max','min'})
    df_1.columns = [f"{col[0]}_{col[1]}" for col in df_1.columns]
    df_1 = df_1.reset_index()
    df = df.merge(df_1, on ='cluster', how = 'left')
    return df 

In [19]:
if CFG.beijing_latlon:
    train_df = beijing_latlon(train_df)
    test_df = beijing_latlon(test_df)

In [20]:
test_df

,date,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,V_10,V_11,V_12,V_13,LAT,LON,power
0,2019/8/20,0.464390,-2.844192,647.669067,303.707458,0.7557,98.452492,0.836825,89,0.350000,3,0.056600,2.058981,237.326815,28.0950,116.9622,31.083333
1,2019/12/12,0.353248,-0.269249,319.740570,278.104065,0.2426,102.246086,0.773377,40,32.419998,13,0.000266,2.604919,76.872996,35.4280,116.6305,61.250000
2,2019/4/7,0.217054,-3.203711,410.029419,295.771332,0.4007,95.362344,0.818272,366,30.200001,12,0.005800,2.404734,235.531159,30.1506,104.6356,26.083333
3,2019/12/2,0.332295,-1.231481,595.750732,285.615051,0.3063,101.249391,0.789055,90,45.349998,13,0.003160,0.060650,159.524072,22.8225,108.3210,23.458333
4,2019/7/14,0.462568,-1.437667,790.363586,304.423859,0.3450,96.466047,0.828973,377,25.780001,12,3.821720,2.643497,318.380560,35.1147,111.0414,29.541667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9745,2019/10/8,0.140074,-1.242913,364.147675,284.673706,0.1685,89.123883,0.789724,1103,42.020000,13,0.000179,0.950499,29.491400,39.3265,112.4078,24.391304
9746,2019/12/19,0.632925,-0.267625,234.627197,276.744598,0.2234,101.986391,0.774328,76,22.379999,13,0.017900,0.523117,81.778249,34.0339,113.8172,98.666667
9747,2019/6/9,0.030754,-2.182944,759.940369,277.601410,0.1740,56.928109,0.778827,4517,11.230000,10,2.434826,4.219044,232.118375,31.4846,92.0657,9.250000
9748,2019/1/24,0.373897,-1.557424,310.455505,283.627258,0.3397,101.131008,0.791993,14,32.740002,13,0.063200,0.804567,55.979027,25.4420,119.0156,54.739130


In [54]:
if CFG.lat_lon:
    train_df = lat_lon(train_df)
    test_df = lat_lon(test_df)

In [55]:
if CFG.corr_features_transform:
    train_df = corr_features_transform(train_df)
    test_df = corr_features_transform(test_df)

In [56]:
if CFG.lag_feats:
    train_df,test_df = lag_feats(train_df,test_df)
    

In [57]:

if CFG.deal_DATE:
    train_df = deal_DATE(train_df)
    test_df = deal_DATE(test_df)

In [58]:
train_df["date"] = pd.to_datetime(train_df["date"])
test_df["date"] = pd.to_datetime(test_df["date"])

In [59]:
if CFG.datetime_features:
    train_df = datetime_features(train_df)
    test_df = datetime_features(test_df)

In [60]:
from sklearn.preprocessing import SplineTransformer
def periodic_spline_transformer(period, n_splines=None, degree=3):
    """
    Kaynak: https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html
    """
    
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True)
def seasonality_spline_features(days=np.arange(1,31)):

    hour_df = pd.DataFrame(np.linspace(1, 31, 31).reshape(-1, 1),columns=["day"])
    splines = periodic_spline_transformer(30, n_splines=60).fit_transform(hour_df)
    splines_df = pd.DataFrame(splines,columns=[f"spline_{i}" for i in range(splines.shape[1])])
    splines_df =pd.concat([pd.Series(days,name='day'), splines_df], axis="columns")
    return splines_df
if CFG.seasonality_spline_features:
    splines_df = seasonality_spline_features()
    train_df = train_df.merge(splines_df,on='day',how='left')
    test_df = test_df.merge(splines_df,on='day',how='left')

In [61]:
if CFG.outlier_detect:
    train_df,test_df = outlier_detect(train_df,test_df)
    

In [62]:
if CFG.extract_features:
    train_df,test_df = extract_features(train_df,test_df)

In [63]:
if CFG.extract_features2:
    train_df,test_df = extract_features2(train_df,test_df)

In [64]:
#train_df['power'] = np.log1p(train_df['power'])
y_train = train_df[CFG.target]
x_train = train_df.drop([CFG.target,"date"],axis=1)
x_test = test_df.drop(["date"], axis=1)

In [65]:
params = {"random_state":42,
          'early_stopping_rounds':1000,
          'iterations': 20000,
        
        'depth': 8,

          }

In [66]:
train_df.shape

(9737, 112)

In [67]:
test_df.shape

(3250, 111)

In [68]:
folds = 5

unseen_preds=[]
models = []
rmses = []


kf = KFold(n_splits=folds, shuffle=True, random_state=42)       



for train_index, valid_index in kf.split(x_train):
    

    x_train_cv = x_train.iloc[train_index]
    y_train_cv = y_train.iloc[train_index]
    x_valid_cv = x_train.iloc[valid_index]
    y_valid_cv = y_train.iloc[valid_index]

    model = CatBoostRegressor(**params)

    model.fit(
        x_train_cv, 
        y_train_cv, 
        eval_set = [(x_train_cv, y_train_cv), (x_valid_cv, y_valid_cv)], 
        #early_stopping_rounds = 1000, 
        verbose = 5000, 
    )
    y_pred_valid = model.predict(x_valid_cv)      
    unseen_preds.append(y_pred_valid)
    temp_rmse_valid = np.sqrt(mean_squared_error(y_valid_cv, y_pred_valid))

    print(f'RMSE(valid_data) = {temp_rmse_valid:.4f}\n')
    

    rmses.append(temp_rmse_valid)

print(f'RMSE(valid_data)_mean = {np.array(rmses).mean():.4f}')
print(f'RMSE(valid_data)_std = {np.array(rmses).std():.4f}')

Learning rate set to 0.011304
0:	learn: 28.0357419	test: 28.0357419	test1: 26.9839757	best: 26.9839757 (0)	total: 48.7ms	remaining: 16m 13s
5000:	learn: 3.3341015	test: 3.3341015	test1: 13.9007295	best: 13.9006457 (4998)	total: 4m 10s	remaining: 12m 32s


KeyboardInterrupt: 

In [ ]:
rmses

In [ ]:
from catboost import CatBoostRegressor


 
 chmaxdepth 8
 
RMSE(valid_data)_mean = 14.5773
RMSE(valid_data)_std = 0.7207

[13.827182563494912,
 15.11508733297057,
 14.118133057214884,
 14.199584963252665,
 15.68433608229355]
 
 RMSE(valid_data)_mean = 14.3108
RMSE(valid_data)_std = 0.4630

[13.822638403808647,
 14.662830535473633,
 14.009918636479004,
 15.040978359183919,
 14.017392007476582]

RMSE(valid_data)_mean = 14.3210
RMSE(valid_data)_std = 0.4777

[13.839101630814174,
 14.701033083826218,
 14.013691499936174,
 15.066938364881509,
 13.984026543793606]
 
 RMSE(valid_data)_mean = 14.2646
RMSE(valid_data)_std = 0.4525
 
 [13.798150662812331,
 14.545167914070186,
 14.013075354136069,
 15.016518198376392,
 13.95027081319474]
 
RMSE(valid_data)_mean = 14.0407
RMSE(valid_data)_std = 0.4400

[13.63963230968322,
 14.366261128422632,
 13.791849659937855,
 14.744383840101472,
 13.661326593564276]
 
 RMSE(valid_data)_mean = 14.0404
RMSE(valid_data)_std = 0.4225

[13.534868984383023,
 14.330993409697266,
 13.881413113384726,
 14.705444799359412,
 13.74919138985025]

In [ ]:
[13.568719867161729,
 14.359696559196909,
 13.847052708323494,
 14.759198888169013,
 13.73156342587145]

In [ ]:
model = CatBoostRegressor(**params)

model.fit(
    x_train, 
    y_train, 
    
    #early_stopping_rounds = 1000, 
    verbose = 5000, 
)
y_pred_valid =  model.predict(x_test)

In [ ]:
#submission["POWER"] = pred
submission["POWER"] = y_pred_valid

In [ ]:
submission

In [ ]:
#submission.to_csv("sub_with_base_features_lgbm_no_hypo.csv",index=False) # 17.11
submission.to_csv("sub_datetime_feats_outlierfeats_add_deal_DATE_spline30_latlontwo_transformfeats.csv",index=False)